# Validation and Quality Checks

Validate transit detections and test reliability.

**Topics:** Parameter validation, injection-recovery, odd-even tests

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from transitkit.core import (
    generate_transit_signal_mandel_agol,
    add_noise,
    find_transits_bls_advanced,
    TransitParameters,
    check_odd_even_consistency,
    check_secondary_eclipse
)
from transitkit.validation import (
    validate_transit_parameters,
    perform_injection_recovery_test
)

## 1. Generate Test Data

In [ ]:
# Create test light curve
time = np.linspace(0, 50, 3500)
flux = generate_transit_signal_mandel_agol(time, period=4.0, t0=2.0, depth=0.012)
flux = add_noise(flux, 0.001, seed=42)

# Detect
result = find_transits_bls_advanced(time, flux)
print(f"Detected: P={result['period']:.4f}d, depth={result['depth']*1e6:.0f}ppm, SNR={result['snr']:.1f}")

## 2. Parameter Validation

In [ ]:
# Create parameters
params = TransitParameters(
    period=result['period'],
    t0=result['t0'],
    depth=result['depth'],
    duration=result['duration'],
    snr=result['snr']
)

# Validate
validation = validate_transit_parameters(params, time, flux)

print('Parameter Validation:')
print('-' * 40)
for key, value in validation.items():
    status = '✓' if value else '✗'
    print(f'  {key}: {status}')

## 3. Odd-Even Test

Check if odd and even transits have consistent depths (eclipsing binary check).

In [ ]:
# Odd-even consistency check
odd_even = check_odd_even_consistency(time, flux, result['period'], result['t0'])

print('Odd-Even Test:')
print(f"  Mean flux (even): {odd_even['mean_even']:.6f}")
print(f"  Mean flux (odd):  {odd_even['mean_odd']:.6f}")
print(f"  p-value:          {odd_even['p_value']:.4f}")
print(f"  Conclusion:       {odd_even['conclusion']}")

## 4. Secondary Eclipse Check

In [ ]:
# Check for secondary eclipse at phase 0.5
secondary = check_secondary_eclipse(time, flux, result['period'], result['t0'])

print('Secondary Eclipse Check:')
print(f"  Primary depth:   {secondary['primary_depth']*1e6:.0f} ppm")
print(f"  Secondary depth: {secondary['secondary_depth']*1e6:.0f} ppm")
print(f"  Depth ratio:     {secondary['secondary_ratio']:.2f}")
print(f"  Detected:        {secondary['detected']}")

## 5. Injection-Recovery Test

Test detection efficiency for different transit depths.

In [ ]:
# Run injection-recovery
print('Running injection-recovery tests (this may take a minute)...\n')

injection_result = perform_injection_recovery_test(
    time,
    injection_params={'period': 4.0, 't0': 2.0, 'duration': 0.12},
    depths=[0.0005, 0.001, 0.002, 0.005, 0.01],
    n_trials=10,
    noise_level=0.001
)

print('Injection-Recovery Results:')
print('-' * 50)
print(f'{"Depth (ppm)":<15} {"Recovery Rate":<15} {"Mean SNR":<15}')
print('-' * 50)
for depth, rate, snr in zip(
    injection_result['depths'],
    injection_result['recovery_rates'],
    injection_result['mean_snrs']
):
    print(f'{depth*1e6:<15.0f} {rate*100:<15.1f}% {snr:<15.1f}')

In [ ]:
# Plot recovery curve
plt.figure(figsize=(8, 5))

depths_ppm = np.array(injection_result['depths']) * 1e6
rates = np.array(injection_result['recovery_rates']) * 100

plt.plot(depths_ppm, rates, 'bo-', ms=8, lw=2)
plt.axhline(50, color='gray', ls='--', label='50% threshold')

plt.xlabel('Transit Depth (ppm)')
plt.ylabel('Recovery Rate (%)')
plt.title('Injection-Recovery: Detection Efficiency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 105)

plt.tight_layout()
plt.show()

## 6. Summary Statistics

In [ ]:
# Summary
print('='*50)
print('VALIDATION SUMMARY')
print('='*50)
print(f"All parameters valid:    {validation.get('all_valid', 'N/A')}")
print(f"Odd-even consistent:     {odd_even['conclusion']}")
print(f"Secondary eclipse:       {'Detected' if secondary['detected'] else 'Not detected'}")
print(f"50% detection threshold: ~{depths_ppm[np.argmin(np.abs(rates-50))]:.0f} ppm")
print('='*50)

## Summary

Validation checks help:
- **Reject false positives**: Eclipsing binaries, systematics
- **Quantify reliability**: Injection-recovery rates
- **Assess significance**: SNR, FAP values

**Best practices:**
- Always run odd-even test
- Check for secondary eclipse
- Report detection efficiency from injection tests